<a href="https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras?scriptVersionId=205427313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<center><h1>Fine-tuning Gemma 2 model using LoRA and Keras</h1></center>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>


# Introduction

This notebook will demonstrate three things:

1. How to fine-tune Gemma model using LoRA
2. Creation of a specialised class to query about Kaggle features
3. Some results of querying about Kaggle Docs

This work is largely based on previous work. Here I list the sources:

1. Gemma 2 Model Card, Kaggle Models,https://www.kaggle.com/models/google/gemma-2/
2. Kaggle QA with Gemma - KerasNLP Starter, Kaggle Code, https://www.kaggle.com/code/awsaf49/kaggle-qa-with-gemma-kerasnlp-starter (Version 11)  
3. Fine-tune Gemma models in Keras using LoRA, Kaggle Code, https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora (Version 1) 
4. Edward J. Hu, Yelong Shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, Weizhu Chen, LoRA: Low-Rank Adaptation of Large Language Models, ArXiv, https://arxiv.org/pdf/2106.09685.pdf
5. Abheesht Sharma, Matthew Watson, Parameter-efficient fine-tuning of GPT-2 with LoRA, https://keras.io/examples/nlp/parameter_efficient_finetuning_of_gpt2_with_lora/
6. Keras 3 API documentation / KerasNLP / Models / Gemma, https://keras.io/api/keras_nlp/models/gemma/
7. Unlock the Power of Gemma 2: Prompt it like a Pro, https://www.kaggle.com/code/gpreda/unlock-the-power-of-gemma-2-prompt-it-like-a-pro  
8. Fine-tune Gemma using LoRA and Keras, https://www.kaggle.com/code/gpreda/fine-tune-gemma-using-lora-and-keras
9. Fine-tunning Gemma model with Kaggle Docs data, https://www.kaggle.com/code/gpreda/fine-tunning-gemma-model-with-kaggle-docs-data
10. Kaggle Docs, Kaggle Dataset, https://www.kaggle.com/datasets/awsaf49/kaggle-docs  


**Let's go**!


# What is Gemma 2?

Gemma is a collection of lightweight, advanced open models developed by Google, leveraging the same research and technology behind the Gemini models. These models are text-to-text, decoder-only large language models available in English, with open weights provided for both pre-trained and instruction-tuned versions. Gemma models excel in a range of text generation tasks, such as question answering, summarization, and reasoning. Their compact size allows for deployment in resource-constrained environments like laptops, desktops, or personal cloud infrastructure, making state-of-the-art AI models more accessible and encouraging innovation for all. 

Gemma 2 represent the 2nd generation of Gemma models. These models were trained on a dataset of text data that includes a wide variety of sources. The **27B** model was trained with **13 trillion** tokens, the **9B** model was trained with **8 trillion tokens**, and **2B** model was trained with **2 trillion** tokens. Here is a summary of their key components: 
* **Web Documents**: A diverse collection of web text ensures the model is exposed to a broad range of linguistic styles, topics, and vocabulary. Primarily English-language content.
* **Code**: Exposing the model to code helps it to learn the syntax and patterns of programming languages, which improves its ability to generate code or understand code-related questions.
* **Mathematics**: Training on mathematical text helps the model learn logical reasoning, symbolic representation, and to address mathematical queries.

To learn more about Gemma 2, follow this link: [Gemma 2 Model Card](https://www.kaggle.com/models/google/gemma-2).




# What is LoRA?  

**LoRA** stands for **Low-Rank Adaptation**. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to **LoRA** paper, this number decreases **10,000 times**, and the computational resources size decreases 3 times. 

# How we proceed?

For fine-tunning with LoRA, we will follow the steps:

1. Install prerequisites
2. Load and process the data for fine-tuning
3. Initialize the code for Gemma causal language model (Gemma Causal LM)
4. Perform fine-tuning
5. Test the fine-tunned model with questions from the data used for fine-tuning and with aditional questions

# Prerequisites


## Install packages

We start by installing `keras-nlp` and `keras` packages.

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -q -U kagglehub --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.6.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.


## Import packages

Now we can import the packages we just installed. We will also install `os`, so that we can set the environment variables needed for keras backend. We will use `jax` as `KERAS_BACKEND`.

Because we want to publish the Model from the Notebook, we also include `kagglehub` and import secrets from `Kaggle App`.

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""
import keras
import keras_nlp
import kagglehub


from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("kaggle_username")
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_key")

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

2024-11-05 17:24:09.650971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 17:24:09.651102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 17:24:09.781847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Configurations


We use a `Config` class to group the information needed to control the fine-tuning process:
* random seed 
* dataset path
* preset - name of pretrained Gemma 2
* sequence length - this is the maximum size of input sequence for training
* batch size - size of the input batch in training, x 2 as two GPUs
* lora rank - rank for LoRA, higher means more trainable parameters 
* learning rate used in the train
* epochs - number of epochs for train

In [3]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/kaggle-docs/questions_answers"
    preset = "gemma2_2b_en" # name of pretrained Gemma 2
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    lora_rank = 5 # rank for LoRA, higher means more trainable parameters
    learning_rate=8e-5 # learning rate used in train
    epochs = 20 # number of epochs to train

Set a random seed for results reproducibility.

In [4]:
keras.utils.set_random_seed(Config.seed)

# Load the data


We load the data we will use for fine-tunining.

In [5]:
df = pd.read_csv(f"{Config.dataset_path}/data.csv")
df.head()

,Question,Answer,Category
0,What are the different types of competitions a...,# Types of Competitions\n\nKaggle Competitions...,competition
1,What are the different competition formats on ...,There are handful of different formats competi...,competition
2,How to join a competition?,"Before you start, navigate to the [Competition...",competition
3,"How to form, manage, and disband teams in a co...",Everyone that competes in a Competition does s...,competition
4,How do I make a submission in a competition?,You will need to submit your model predictions...,competition


Let's check the total number of rows in this dataset.

In [6]:
df.shape[0]

60

For easiness, we will create the following template for QA: 

In [7]:
template = "\n\nCategory:\nkaggle-{Category}\n\nQuestion:\n{Question}\n\nAnswer:\n{Answer}"
df["prompt"] = df.apply(lambda row: template.format(Category=row.Category,
                                                             Question=row.Question,
                                                             Answer=row.Answer), axis=1)
data = df.prompt.tolist()

## Template utility function

In [8]:
def colorize_text(text):
    for word, color in zip(["Category", "Question", "Answer"], ["blue", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Specialized class to query Gemma


We define a specialized class to query Gemma. But first, we need to initialize an object of GemmaCausalLM class.

## Initialize the code for Gemma Causal LM

In [9]:
gemma_causal_lm = keras_nlp.models.GemmaCausalLM.from_preset(Config.preset)
gemma_causal_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

## Define the specialized class

Here we define the special class `GemmaQA`. 
in the `__init__` we pass the `GemmaCausalLM` object created before.
The `query` member function uses `GemmaCausalLM` member function `generate` to generate the answer, based on a prompt that includes the category and the question.

In [10]:
class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_causal_lm = gemma_causal_lm
        
    def query(self, category, question):
        response = self.gemma_causal_lm.generate(
            self.prompt.format(
                Category=category,
                Question=question,
                Answer=""), 
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))
        

## Gemma preprocessor


This preprocessing layer will take in batches of strings, and return outputs in a ```(x, y, sample_weight)``` format, where the y label is the next token id in the x sequence.

From the code below, we can see that, after the preprocessor, the data shape is ```(num_samples, sequence_length)```.

In [11]:
x, y, sample_weight = gemma_causal_lm.preprocessor(data[0:2])

In [12]:
print(x, y)

{'token_ids': Array([[     2,    109,   8606, ...,  25688, 235290,  75676],
       [     2,    109,   8606, ...,    109,    688,   2299]],      dtype=int32), 'padding_mask': Array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)} [[   109   8606 235292 ... 235290  75676      1]
 [   109   8606 235292 ...    688   2299      1]]


# Perform fine-tuning with LoRA

## Enable LoRA for the model

LoRA rank is setting the number of trainable parameters. A larger rank will result in a larger number of parameters to train.

In [13]:
# Enable LoRA for the model and set the LoRA rank to the lora_rank as set in Config (4).
gemma_causal_lm.backbone.enable_lora(rank=Config.lora_rank)
gemma_causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,618,002,688 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,618,002,688 (9.75 GB)

 Trainable params: 3,660,800 (13.96 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

We see that only a small part of the parameters are trainable. 2.6 billions parameters total, and only 2.9 Millions parameters trainable.

## Run the training sequence

We set the `sequence_length` for the `GemmaCausalLM` (from configuration, will be 512).
We compile the model, with the loss, optimizer and metric.
For the metric, it is used `SparseCategoricalAccuracy`. This metric calculates how often predictions match integer labels.

In [14]:
#set sequence length cf. config (512)
gemma_causal_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_causal_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 88s 828ms/step - loss: 1.6834 - sparse_categorical_accuracy: 0.5354
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 825ms/step - loss: 1.5878 - sparse_categorical_accuracy: 0.5486
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 826ms/step - loss: 1.5177 - sparse_categorical_accuracy: 0.5575
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 826ms/step - loss: 1.4742 - sparse_categorical_accuracy: 0.5677
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - loss: 1.4314 - sparse_categorical_accuracy: 0.5768
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - loss: 1.3843 - sparse_categorical_accuracy: 0.5854
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 826ms/step - loss: 1.3279 - sparse_categorical_accuracy: 0.5973
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 826ms/step - loss: 1.2613 - sparse_categorical_accuracy: 0.6115
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 825ms/step - loss: 1.1836 - sparse_categorical_accuracy: 0.6291
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 826ms/ste

# Test the fine-tuned model

We instantiate an object of class GemmaQA. Because `gemma_causal_lm` was fine-tuned using LoRA, `gemma_qa` defined here will use the fine-tuned model.

In [15]:
gemma_qa = GemmaQA()

For start, we are testing the model with some of the data from the training set itself.

## Sample 1

In [16]:
row = df.iloc[0]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-competition

**<font color='red'>Question:</font>**
What are the different types of competitions available on Kaggle?

**<font color='green'>Answer:</font>**
# Types of Competitions

Kaggle Competitions are a central part of the Kaggle community’s collaborative data science problem-solving ethos. Every week, a new competition is launched with the goal of tackling a real-world data science or machine learning problem in just one week.

Currently, Kaggle Competitions are launched on a weekly basis. In the future, we plan to increase the cadence to launch a new competition every other day or even more frequently.

Currently, Kaggle Competitions are launched in two formats:

# 1) Community Competitions

Community Competitions are launched with a relatively open problem statement and encourage collaborative problem-solving. Community Competition participants have the freedom to explore any approach they’d like and may or may not use the public data or ground truth solution provided. At the end of the competition period, the team with the highest overall accuracy on the test set will be declared the winner.

If you’d like to encourage collaboration and foster a friendly problem-solving atmosphere, Community Competitions are a great choice.

# 2) Restricted Competitions

Restricted Competitions are launched with a more specific problem statement and are restricted to teams that register and accept the competition’s terms and conditions. This type of competition is a good fit for problems that require restricted data or an exclusive solution set.

## Examples

Here are some examples of the different types of Kaggle Competitions we launch:

### Community Competitions

[COVID-19 Hospitalizations Prediction Competition](https://www.kaggle.com/c/covid19-hospitalizations-prediction)

[Titanic: Machine Learning from Disaster Competition](https://www.kaggle.com/c/the-titanic-machine-learning-from-disaster)

[2020 Super Mario Maker Competition](https://www.kaggle.com/c/super-mario-maker-dataset)

### Restricted Competitions

[2019 Leaderboard Competition](https://www.kaggle.com/c/2019-leaderboard-competition)

[2018 Leaderboard Competition](https://www.kaggle.com/c/2018-leaderboard-competition)

[2017 Leaderboard Competition](https://www.kaggle.com/c/2017-leaderboard-competition)

[2016 Leaderboard Competition](https

## Sample 2

In [17]:
row = df.iloc[15]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-tpu

**<font color='red'>Question:</font>**
How to load and save model on TPU?

**<font color='green'>Answer:</font>**
When saving and loading a model to/from disk on TPU, we need to use special functionality due to the way TPUs are organized.

First, we need to create a TPUPostV2. This is a special VM that TPUs run on. It has a lot of extra features, including saving and loading functionality for models in TensorFlow’s model.json format. To create a TPUPostV2, run the command `tpu-proxmox-image`.

Now that we have a TPUPostV2, we can create a TPUSession from it. The arguments for creating a TPUSession are:

```
TPUSession(tpu-proxmox-image) # creates a TPUPostV2 VM from Proxmox
TPUSession(p

## Sample 3

In [18]:
row = df.iloc[25]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-noteboook

**<font color='red'>Question:</font>**
What are the different types of notebooks available on Kaggle?

**<font color='green'>Answer:</font>**
There are two different types of notebooks available on Kaggle: Jupiter Notebooks and Python Scripts.

## Jupiter Notebooks

Jupiter Notebooks are a special type of notebook that Kaggle provides. They are based on the open source data science platform Jupiter from Google. Jupiter Notebooks allow seamless collaboration with comments and versioning, and they integrate with Kaggle competitions and other features of the Kaggle platform.

Jupiter Notebooks are written in Python and use the Jupiter Notebook extension for TensorFlow (v1.13 or later) or Keras (v2.1.0 or later) libraries for deep learning. For other machine learning libraries, Jupiter Notebooks typically use common libraries like pandas, scikit-learn, or TensorFlow Probability.

When creating a Notebook from the New Notebook button, Kaggle allows you to select templates for common use cases like data exploration, model training, or

## Not seen question(s)

In [19]:
category = "notebook"
question = "How to run a notebook?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-notebook

**<font color='red'>Question:</font>**
How to run a notebook?

**<font color='green'>Answer:</font>**
When you run a notebook, its cells are executed one after another, from top to
bottom. The execution state of a cell (i.e., whether it succeeded or failed)
communicates the execution state of the cells below it using "warming pan"
syntax. For example, a cell that throws an error will make all the cells
below it invalid (the "cold" state). Conversely, a successful cell makes
below it valid, with the possible exception of the last cell which validity
depends on the specific execution strategy.

When you run a notebook from the Notebook interface, you can pause a
execution at any time using the keyboard's pause key (spacebar). You can
resume from the same position by pressing the spacebar again.

If you prefer to control execution manually from the command line, KubeShell
provides an `execute` command for executing notebook commands. For example:

```
kagub `execute notebook --name kaggle-samples --command execute --name-to-name-mapping {'Run': 'run'} syntax to execute lines one after another, another time to execute the entire notebook
```

If you prefer to control execution manually one

In [20]:
category = "discussions"
question = "How to create a discussion topic?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-discussions

**<font color='red'>Question:</font>**
How to create a discussion topic?

**<font color='green'>Answer:</font>**
Discussions is a threaded view of interesting conversations in machine learning. To start a conversation, head to [Discussions > Community](https://www.kaggle.com/discussions/community) or any of the other notebooks view formats and click on New Discussion in the upper-right corner.

First provide a title that describes the topic of your discussion. Then jump into the text editor to share your thoughts.

If you are posting a question, consider breaking it up into smaller questions if it gets too long. It is better to have multiple shorter questions that people can answer than one long question that no one can.

When you are finished with your post, mark it up or down the community posts that are interesting to you. This helps identify the types of conversations that are most valuable to have on the platform.

When you hover over a community post, you will see reaction buttons just like on an puzzle.

If you click on “Report Post”, a modal will open where you can select the reason for your report.

If you have any feedback or issues with a discussion, you can report the post in the same way as with individual puzzle files.

When creating a discussion you will also notice an “subscribe” button. Subscribing to a discussion will add it to the right-hand side of the page in the “Recently Added” view format. This way you can quickly review new discussions that have been posted.

## Viewing Replies

By default, when you click on a new discussion, you will view the main post and the discussions replies on the same page. To view replies one after another, click on “Next” at the bottom of the page after each reply.

If you want to view replies in a separate window, from the same page you can also click on “Discussion Replies List”.

## Discussion Replies List

From the same page as before, you can also access a separate window for discussion replies list view. This is an option if you prefer to view replies in a separate window from the main discussion page or if you plan to jump between discussions frequently.

##

In [21]:
category = "competitions"
question = "What is a code competition?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-competitions

**<font color='red'>Question:</font>**
What is a code competition?

**<font color='green'>Answer:</font>**
Code competitions are events where participants are challenged to solve a series of problems using a programming language of their choice. The problems are often open-ended, allowing for creative solutions, but they may also be more structured, with specific requirements or constraints.

At the end of the competition period, participants submit their solutions and the organizers will provide a scoring or evaluation mechanism. The top-performing solutions will be recognized and awarded prizes or other recognition.

Code competitions can be a great way to improve your programming skills, meet other programmers, and work on challenging problems in a competitive environment. They are a popular form of programming engagement and are offered by Kaggle and other organizations.

In [22]:
category = "datasets"
question = "What are the steps to create a Kaggle dataset?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-datasets

**<font color='red'>Question:</font>**
What are the steps to create a Kaggle dataset?

**<font color='green'>Answer:</font>**
You can create a new Kaggle dataset in two ways:

# Using the web interface
Click the `Create new content...` button at the top of any dataset's page. You will be taken to a modal that allows you to enter the details of your dataset.

- Enter a descriptive title and subtitle for your dataset in the top fields.
- Select the `Dataset` content type from the menu options.
- Check the "Share with public" box to make the dataset publicly viewable.
- Upload a README file from your computer (or paste in the URL) to a folder on your desktop. The folder will be your dataset's public folder.
- Optionally, upload more files from your computer into the "Files" section below the file upload field. These files will be from your desktop and will be visible to anyone with a link. If you need to upload more files that are not from your desktop, use the technique described in the next section.
- Select the "Permissions" tab and, for the users listed here, select the "View" option. Datasets created with public permissions will not

# Save the model

In [23]:
preset_dir = ".\gemma2_2b_en_kaggle_docs"
gemma_causal_lm.save_to_preset(preset_dir)

# Publish Model on Kaggle as a Kaggle Model

We are publishing now the saved model as a Kaggle Model.

In [24]:
kaggle_username = os.environ["KAGGLE_USERNAME"]

kaggle_uri = f"kaggle://{kaggle_username}/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

Uploading Model https://www.kaggle.com/models/gpreda/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs ...
Starting upload for file .\gemma2_2b_en_kaggle_docs/tokenizer.json


Uploading: 100%|██████████| 591/591 [00:00<00:00, 873B/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/tokenizer.json (591B)
Starting upload for file .\gemma2_2b_en_kaggle_docs/metadata.json



Uploading: 100%|██████████| 143/143 [00:00<00:00, 215B/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/metadata.json (143B)
Starting upload for file .\gemma2_2b_en_kaggle_docs/preprocessor.json


...


Uploading: 100%|██████████| 10.5G/10.5G [03:07<00:00, 55.9MB/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/model.weights.h5 (10GB)
Starting upload for file .\gemma2_2b_en_kaggle_docs/assets/tokenizer/vocabulary.spm



Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 5.85MB/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/assets/tokenizer/vocabulary.spm (4MB)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/gpreda/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs


# Conclusions



We demonstated how to fine-tune a **Gemma 2** model using LoRA.   
We also created a class to run queries to the **Gemma 2** model and tested it with some examples from the existing training data but also with some new, not seen questions.   
We also saved the models as a Keras model. 
Then we published the model as a Kaggle Model on Kaggle Models platform.